# Introduction to _py2cytoscape_
#### Pythonista-friendly wrapper for cyREST

### What is this?
From version 0.4.0, ___py2cytoscape___ has wrapper for [cyREST](http://apps.cytoscape.org/apps/cyrest) RESTful API.  This means you can access Cytoscape features in more _Pythonic_ way instead of calling raw REST API via HTTP.  Since [pandas](http://pandas.pydata.org/) is a standard library for data mangling/analysis in Python, this new version uses its [DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#pandas.DataFrame) as its basic data object.

#### Questions or Feature Request
Send me an email (kono at ucsd).

In [1]:
from py2cytoscape.data.cynetwork import CyNetwork
from py2cytoscape.data.cyrest_client import CyRestClient
import py2cytoscape.util.cytoscapejs as cyjs

import networkx as nx
import pandas as pd

# Create an instance of cyREST client.  Default IP is 'localhost', and port number is 1234.
# cy = CyRestClient() - This default constructor creates connection to http://localhost:1234/v1
cy = CyRestClient(ip='127.0.0.1', port=1234)

# Cleanup
cy.network.delete_all()

## Creating empty networks
Creation will be done by the cyREST client.

In [2]:
# Empty network
empty1 = cy.network.create()

# With name
empty2 = cy.network.create(name='Created in Jupyter Notebook')

# With name and collection name
empty3 = cy.network.create(name='Also created in Jupyter', collection='New network collection')

## Create networks from various types of data
Currently, py2cytoscape accepts the following data as input:

* Cytoscape.js
* NetworkX
* Pandas DataFrame
* igraph
* Numpy adjacency matrix (binary or weighted)
* GraphX (TBD)

In [3]:
# Cytoscape.js JSON
n1 = cy.network.create(data=cyjs.get_empty_network(), name='Created from Cytoscape.js JSON')

# Pandas DataFrame
df_from_sif = pd.read_csv('../tests/data/galFiltered.sif', names=['source', 'interaction', 'target'], sep=' ')
yeast1 = cy.network.create_from_dataframe(df_from_sif, name='Yeast network created from pandas DataFrame')

# NetworkX
nx_graph  = nx.scale_free_graph(100)
nx.set_node_attributes(nx_graph, 'Degree', nx.degree(nx_graph))
nx.set_node_attributes(nx_graph, 'Betweenness_Centrality', nx.betweenness_centrality(nx_graph))
scale_free100 = cy.network.create_from_networkx(nx_graph)

# TODO: igraph
# TODO: Numpy adj. martix
# TODO: GraphX

## Working with CyNetwork API
___CyNetwork___ class is a simple wrapper to cyREST raw REST API.  __It does not hold the actual network data.  It's a reference to a network in current Cytoscape session__.  With CyNetwork API, you can access Cytoscape data objects in more Pythonista-friendly way.

In [4]:
network_suid = yeast1.get_id()
print('This object references to Cytoscape network with SUID ' + str(network_suid) + '\n')
print('And its name is: ' + yeast1.get_network_value(column='name') + '\n')

nodes = yeast1.get_nodes()

# Get a row in the node table as pandas Series object
node0 = nodes[0]
row = yeast1.get_node_value(id=node0)
print(row)

# Or, pick one cell in the table
cell = yeast1.get_node_value(id=node0, column='name')
print('\nThis node has name: ' + cell) 

This object references to Cytoscape network with SUID 108649

And its name is: Yeast network created from pandas DataFrame

SUID            108659
id             YKR026C
name           YKR026C
selected         False
shared_name    YKR026C
dtype: object

This node has name: YKR026C


### Get references to existing networks
And of course, you can grab references to existing Cytoscape networks:

In [5]:
# Create a new CyNetwork object from existing network
network_ref1 = cy.network.create(suid=yeast1.get_id())

# And they are considered as same objects.
print(network_ref1 == yeast1)
print(network_ref1.get_network_value(column='name'))

True
Yeast network created from pandas DataFrame


----
## Network Topology Modification
(TBD)

----

## Tables as DataFrame
Cytoscape has two main data types: ___Network___ and ___Table___.  Network is the graph topology, and Tables are properties for those graphs.  For simplicity, this library has access to three basic table objects:

* Node Table
* Edge Table
* Network Table

For 99% of your use cases, you can use these three to store properties.  Since [pandas](http://pandas.pydata.org/) is extremely useful to handle table data, default data type for tables is __DataFrame__.  However, you can also use other data types including:

* Cytoscape.js style JSON
* CSV
* TSV
* CX (TBD)

In [6]:
# Get table  from Cytoscape
node_table = scale_free100.get_node_table()
edge_table = scale_free100.get_edge_table()
network_table = scale_free100.get_network_table()

In [7]:
node_table.head()

,shared name,id,name,selected,Degree,Betweenness_Centrality
SUID,,,,,,
110057,0,0,0,False,48,0.053503
110058,1,1,1,False,28,0.057024
110059,2,2,2,False,69,0.047473
110060,3,3,3,False,18,0.013674
110061,4,4,4,False,6,0.002680


In [8]:
network_table.transpose().head()

SUID,110047
shared name,"directed_scale_free_graph(100,alpha=0.41,beta=..."
name,"directed_scale_free_graph(100,alpha=0.41,beta=..."
selected,True
__Annotations,NaN


## Update Table
Let's do something a bit more realistic.  You can update any Tables by using DataFrame objects.

### 1. ID conversion with external service
Let's use [ID Conversion web service by Uniprot](http://www.uniprot.org/help/programmatic_access) to add more information to existing yeast network in current session.

In [9]:
# Small utility function to convert ID sets
import requests
import StringIO

def uniprot_id_mapping_service(query=None, from_id=None, to_id=None):
    # Uniprot ID Mapping service
    url = 'http://www.uniprot.org/mapping/'
    payload = {
        'from': from_id,
        'to': to_id,
        'format':'tab',
        'query': query
    }
    res = StringIO.StringIO(requests.get(url, params=payload).content)
    df = pd.read_csv(res, sep='\t')
    res.close()
    return df

In [15]:
# Get node table from Cytoscape
yeast_node_table = yeast1.get_node_table()

# From KEGG ID to UniprotKB ID
query1 = ' '.join(yeast_node_table['name'].map(lambda gene_id: 'sce:' + gene_id).values)
id_map_kegg2uniprot = uniprot_id_mapping_service(query1, from_id='KEGG_ID', to_id='ID')
id_map_kegg2uniprot.columns = ['kegg', 'uniprot']

# From UniprotKB to SGD
query2 = ' '.join(id_map_kegg2uniprot['uniprot'].values)
id_map_uniprot2sgd = uniprot_id_mapping_service(query2, from_id='ID', to_id='SGD_ID')
id_map_uniprot2sgd.columns = ['uniprot', 'sgd']

# From UniprotKB to Entrez Gene ID
query3 = ' '.join(id_map_kegg2uniprot['uniprot'].values)
id_map_uniprot2ncbi = uniprot_id_mapping_service(query3, from_id='ID', to_id='P_ENTREZGENEID')
id_map_uniprot2ncbi.columns = ['uniprot', 'entrez']

# Merge them
merged = pd.merge(id_map_kegg2uniprot, id_map_uniprot2sgd, on='uniprot')
merged = pd.merge(merged, id_map_uniprot2ncbi, on='uniprot')

# Add key column by removing prefix
merged['name'] = merged['kegg'].map(lambda kegg_id : kegg_id[4:])
merged.head()

,kegg,uniprot,sgd,entrez,name
0,sce:YKR026C,EI2BA_YEAST,S000001734,853896,YKR026C
1,sce:YGL122C,NAB2_YEAST,S000003090,852755,YGL122C
2,sce:YGR218W,XPO1_YEAST,S000003450,853133,YGR218W
3,sce:YGL097W,RCC1_YEAST,S000003065,852782,YGL097W
4,sce:YOR204W,DED1_YEAST,S000005730,854379,YOR204W


In [11]:
# Now update existing node table with the data frame above.
yeast1.update_node_table(merged, network_key_col='name', data_key_col='name')

# Check the table is actually updated
yeast1.get_node_table().head()

,shared name,name,selected,id,Degree,Betweenness_Centrality,kegg,entrez,sgd,uniprot
SUID,,,,,,,,,,
108659,YKR026C,YKR026C,False,YKR026C,NaN,NaN,sce:YKR026C,853896,S000001734,EI2BA_YEAST
108660,YGL122C,YGL122C,False,YGL122C,NaN,NaN,sce:YGL122C,852755,S000003090,NAB2_YEAST
108661,YGR218W,YGR218W,False,YGR218W,NaN,NaN,sce:YGR218W,853133,S000003450,XPO1_YEAST
108662,YGL097W,YGL097W,False,YGL097W,NaN,NaN,sce:YGL097W,852782,S000003065,RCC1_YEAST
108663,YOR204W,YOR204W,False,YOR204W,NaN,NaN,sce:YOR204W,854379,S000005730,DED1_YEAST


## Create / Delete Table Data
Currently, ___you cannot delete the table or rows___ due to the Cytoscape data model design.  However, it is easy to create / delete columns:

In [12]:
# Delete columns
yeast1.delete_node_table_column('Degree')
yeast1.delete_node_table_column('Betweenness_Centrality')

# Create columns
yeast1.create_node_column(name='New Empty Double Column', data_type='Double', is_immutable=False, is_list=False)

# Default is String, mutable column.
yeast1.create_node_column(name='Empty String Col')

yeast1.get_node_table().head()

,shared name,name,selected,id,kegg,entrez,sgd,uniprot,New Empty Double Column,Empty String Col
SUID,,,,,,,,,,
108659,YKR026C,YKR026C,False,YKR026C,sce:YKR026C,853896,S000001734,EI2BA_YEAST,NaN,NaN
108660,YGL122C,YGL122C,False,YGL122C,sce:YGL122C,852755,S000003090,NAB2_YEAST,NaN,NaN
108661,YGR218W,YGR218W,False,YGR218W,sce:YGR218W,853133,S000003450,XPO1_YEAST,NaN,NaN
108662,YGL097W,YGL097W,False,YGL097W,sce:YGL097W,852782,S000003065,RCC1_YEAST,NaN,NaN
108663,YOR204W,YOR204W,False,YOR204W,sce:YOR204W,854379,S000005730,DED1_YEAST,NaN,NaN


## Visual Styles
(TBD)

In [13]:
# style1 = cy.style.create('My Style')
# style1.create_descrete_mapping(data_type='node', column='type', visual_property='NODE_SHAPE', mapping=dict_map)

## Algorithmic resources
(TBD)

In [14]:
# Apply layout
# cy.layout.apply('layout_algorithm_name', network)